Library Imports

In [3]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
#import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [26]:
#Import Data
etfData = pd.read_csv('Data\ETFData_min.csv',parse_dates=['datetime'])
etfData.sort_values(by='datetime', ascending=True, inplace=True)
etfData

,datetime,510300.SH,588000.SH,159869.SZ,159915.SZ,510500.SH,510880.SH,512010.SH,512100.SH,512200.SH,...,516970.SH,159611.SZ,159667.SZ,159745.SZ,159870.SZ,159996.SZ,512720.SH,515250.SH,515400.SH,516110.SH
0,2023-01-03 09:30:00,4.557921,1.001,0.759,2.264,1.888272,4.308541,1.870490,0.910284,0.720,...,1.060,0.863,0.909,0.775,0.751,1.004,1.058,0.775,0.741,0.961
1,2023-01-03 09:31:00,4.556757,1.002,0.763,2.263,1.890519,4.310046,1.878300,0.912820,0.722,...,1.061,0.864,0.907,0.776,0.749,1.007,1.056,0.776,0.746,0.960
2,2023-01-03 09:32:00,4.559086,1.001,0.762,2.262,1.891482,4.308541,1.870490,0.912095,0.722,...,1.062,0.866,0.908,0.777,0.749,1.007,1.057,0.776,0.745,0.963
3,2023-01-03 09:33:00,4.561414,1.002,0.765,2.262,1.890198,4.310046,1.862680,0.911733,0.720,...,1.061,0.866,0.910,0.777,0.753,1.008,1.055,0.777,0.746,0.965
4,2023-01-03 09:34:00,4.559086,1.004,0.760,2.263,1.891482,4.308541,1.862680,0.912820,0.718,...,1.061,0.868,0.912,0.776,0.753,1.007,1.055,0.778,0.746,0.964
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40167,2023-09-06 14:56:00,4.591570,0.978,1.139,2.049,1.892446,4.809309,1.608856,0.902315,0.657,...,1.151,0.887,0.965,0.754,0.673,1.130,1.156,0.872,0.860,1.067
40168,2023-09-06 14:57:00,4.591570,0.978,1.139,2.049,1.892124,4.807729,1.604951,0.902315,0.657,...,1.151,0.887,0.964,0.754,0.673,1.130,1.156,0.871,0.860,1.068
40169,2023-09-06 14:58:00,4.591570,0.978,1.139,2.049,1.891803,4.807729,1.608856,0.902315,0.657,...,1.150,0.887,0.964,0.754,0.673,1.130,1.156,0.872,0.859,1.068
40170,2023-09-06 14:59:00,4.592752,0.979,1.139,2.049,1.892446,4.804568,1.608856,0.902315,0.657,...,1.151,0.887,0.964,0.754,0.673,1.130,1.156,0.871,0.859,1.066


In [10]:
etfList = pd.read_csv('Data\ETFList.csv',header = None)
col_names = ['ETFName']
etfList.columns = col_names
etfList

,ETFName
0,510300.SH
1,588000.SH
2,159869.SZ
3,159915.SZ
4,510500.SH
5,510880.SH
6,512010.SH
7,512100.SH
8,512200.SH
9,512480.SH


In [37]:
#Log message function
def log(msg):
    timestamp = datetime.datetime.utcnow().strftime("%b %d %Y %H:%M:%S UTC")
    s = "[%s] %s:%s %s" % (timestamp, COLOR_WHITE, COLOR_RESET, msg)
    print(s)
    try:
        f = open(LOGFILE, "a")
        f.write(s + "\n")
        f.close()
    except:
        pass

def initialize_result():
    s = f'Time, Grid Level, ETF On Hold, Cash, ETF Price, ETF Asset, Total Asset'
    print(s)
    try:
        f = open(RESULT_FILE, "w")
        f.write(s + "\n")
        f.close()
    except:
        print("Error in result file initlization")

def write_result():
    timestamp = datetime.datetime.utcnow().strftime("%b %d %Y %H:%M:%S UTC")
    s = f'Time, Grid Level, ETF On Hold, Cash, ETF Price, ETF Asset, Total Asset'
    print(s)
    try:
        f = open(RESUlT_FILE, "a")
        f.write(s + "\n")
        f.close()
    except:
        print("Error in result file write")

In [53]:
def grid_price(level):
    return basePrice * (1 + INTERVAL) ** (level - GRID_LEVELS / 2)

In [ ]:
def place_order(level):
    if 0 <= level <= GRID_LEVELS:
        #number of stock to purchase
        if (level > Current_Level): #Purchase
            cash_to_use = Current Cash - Current_Cash/(GRID_LEVELS - Current_Level)*(GRID_LEVELS - level) 
            stock_to_purchase = round(cash_to_use / Current_Price / 100) * 100 
            cash_to_use = stock_to_purchase * Current_Price + stock_to_purchase * Current_Price * ETF_ORDER_FEE
    else:
        log（f'Invalid level. Level: {Current_Level}'

In [40]:
#Initializa Variables

#Files
LOG_FILE = 'Output/logfile.txt' #Log file
RESULT_FILE = 'Output/result.txt' #Monitor all price changes

#Variables
GRID_LEVELS = 10 #网格数量
GRID_INCREMENT_TYPE = 'Ratio' #网格增加减小方式 Ratio 或者 Number
GRID_TYPE = 'cash' #网格基于资金还是ETF份数 cash or ETF
GRID_INTERVAL = 0.01 #网格最大最小值

ETF_ORDER_FEE = 0.003 #交易手续费
TOTAL_FUND = 1000000 #原始资金
SYMBOL = "510300.SH" #本次回测使用的ETF代码

#Initialize Result File
initialize_result()

Time, Grid Level, ETF On Hold, Cash, ETF Price, ETF Asset, Total Asset


In [55]:
#当前Symbol的分钟数据
Current_ETF_Data = etfData[['datetime', SYMBOL]]

#初始化基础仓位
Base_Price = Current_ETF_Data.iloc[0][SYMBOL] #取第一个价格作为基础价格

Previous_Level = Current_Level = 0
Next_Level_Price_Up = 0.0
Next_Level_Price_Down = 0.0
Level_To_Set = round(GRID_LEVELS / 2) #当前网格 0 - GRID_LEVELS
Current_Stock = 0 #当前持股数
Current_Price = Base_Price #当前价格
Current_Cash = TOTAL_FUND #当前现金

#place_order(level = Currnet_Level_To_Set) 




NameError: name 'Next_Level_Price_Up' is not defined